# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [1]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='sga')

from pyspark.sql import SparkSession, Row
se = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-10-26 12:26:31,474 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [62]:
import numpy as np
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [89]:
clickstream = se.read.csv("hdfs:/data/clickstream.csv", sep='\t', header=True)
clickstream.registerTempTable('clickstream')

In [90]:
clickstream.show()

+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1695584127|
|    562|       507|       event|      main|1695584134|
|    562|       507|       event|      main|1695584144|
|    562|       507|       event|      main|1695584147|
|    562|       507|wNaxLlerrorU|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584160|
|    562|       507|        page|    rabota|1695584166|
|    562|       507|       event|    rabota|1695584174|
|    562|       507|       event|    rabota|1695584181|
|    562|       507|       event|    rabota|1695584189|
|    562|       507|        page|      main|1695584194|
|    562|       507|       event|      main|1695584204|
|    562|       507|       event|      main|1695

### Solution with Spark SQL

In [43]:
# filter data to get lists of routes without errors and get rid of repeated pages
filter_data_query = """
        with clickstream_with_flag as(
            select
                    *,
                    max(case when event_type like "%error%" then 1 else 0 end) over (partition by user_id, session_id order by timestamp) as error,
                    lag(event_page) over (partition by user_id, session_id order by timestamp) as previous_page
            from clickstream
        )
        select
                user_id,
                session_id,
                row_number() over (partition by user_id, session_id ORDER BY timestamp desc) as row_num,
                collect_list(event_page) over (partition by user_id, session_id ORDER BY timestamp) as route
        from clickstream_with_flag
        where error = 0
                and (previous_page != event_page or previous_page is null)
        """
filtered_table = se.sql(filter_data_query)
filtered_table.registerTempTable('filtered_table')
filtered_table.show()

+-------+----------+-------+--------------------+
|user_id|session_id|row_num|               route|
+-------+----------+-------+--------------------+
|      1|      1026|      1|              [main]|
|     10|       762|     12|              [main]|
|     10|       762|     11|    [main, internet]|
|     10|       762|     10|[main, internet, ...|
|     10|       762|      9|[main, internet, ...|
|     10|       762|      8|[main, internet, ...|
|     10|       762|      7|[main, internet, ...|
|     10|       762|      6|[main, internet, ...|
|     10|       762|      5|[main, internet, ...|
|     10|       762|      4|[main, internet, ...|
|     10|       762|      3|[main, internet, ...|
|     10|       762|      2|[main, internet, ...|
|     10|       762|      1|[main, internet, ...|
|   1001|        11|      4|              [main]|
|   1001|        11|      3|       [main, bonus]|
|   1001|        11|      2|[main, bonus, int...|
|   1001|        11|      1|[main, bonus, int...|


In [44]:
# concatenate lists into routes of type page-page-page
result_query = """
                    select route_concat,
                            count(session_id) as count
                    from (
                            select user_id,
                                    session_id,
                                    concat_ws("-", route) as route_concat
                            from filtered_table
                            where row_num = 1
                    ) as tmp
                    group by route_concat
                    order by 2 desc
                """

result_sql = se.sql(result_query)
result_sql.show(30)

+--------------------+-----+
|        route_concat|count|
+--------------------+-----+
|                main| 8184|
|        main-archive| 1113|
|         main-rabota| 1047|
|       main-internet|  897|
|          main-bonus|  870|
|           main-news|  769|
|        main-tariffs|  677|
|         main-online|  587|
|          main-vklad|  518|
| main-rabota-archive|  170|
| main-archive-rabota|  167|
|  main-bonus-archive|  143|
|   main-rabota-bonus|  139|
|    main-news-rabota|  135|
|   main-bonus-rabota|  135|
|main-archive-inte...|  132|
|    main-rabota-news|  130|
|main-internet-rabota|  129|
|   main-archive-news|  126|
|main-rabota-internet|  124|
|main-internet-arc...|  123|
|  main-archive-bonus|  117|
| main-internet-bonus|  115|
|main-tariffs-inte...|  114|
|   main-news-archive|  113|
|  main-news-internet|  109|
|main-archive-tariffs|  104|
|  main-internet-news|  103|
|main-tariffs-archive|  103|
|    main-rabota-main|   94|
+--------------------+-----+
only showing t

In [148]:
temp = result_sql.limit(30).toPandas()
with open('result_sql.csv', 'w') as f:
    f.write(temp.to_csv(sep='\t', index=False, header=['route', 'count']))

In [52]:
result_grader.loc[0].to_list()

['main', 8184]

In [58]:
result = {}
for i in range(0, 10):
    line = result_grader.loc[i].to_list()
    result[line[0]] = int(line[1])

In [56]:
result

{'main': 8184,
 'main-archive': 1113,
 'main-rabota': 1047,
 'main-internet': 897,
 'main-bonus': 870,
 'main-news': 769,
 'main-tariffs': 677,
 'main-online': 587,
 'main-vklad': 518,
 'main-rabota-archive': 170}

### Solution with Spark DF

In [84]:
window = Window.partitionBy('user_id', 'session_id')

filtered_table = (
    clickstream
                  .withColumn('error', F.max(
                                          F.when(clickstream.event_type.like('%error%'), 1).otherwise(0)
                                              ).over(window.orderBy(clickstream.timestamp)))
                 .withColumn('previous_page', F.lag(
                                                     clickstream.event_page
                                                 ).over(window.orderBy(clickstream.timestamp)))
                .filter('error = 0 and (previous_page != event_page or previous_page is null)')
                  .withColumn('row_num', F.row_number().over(window.orderBy(F.desc(clickstream.timestamp))))
                  .withColumn('route_list', F.collect_list(clickstream.event_page)
                                             .over(window.orderBy(clickstream.timestamp)))
                .withColumn('route', F.concat_ws('-', 'route_list'))
                .filter('row_num = 1')
                .select(['user_id', 'session_id', 'route'])
)
filtered_table.show()

+-------+----------+--------------------+
|user_id|session_id|               route|
+-------+----------+--------------------+
|      1|      1026|                main|
|     10|       762|main-internet-arc...|
|   1001|        11|main-bonus-intern...|
|   1002|       285|main-news-interne...|
|   1002|       291|        main-archive|
|   1003|       563|main-internet-mai...|
|   1003|       593|main-internet-onl...|
|   1004|       871|main-archive-vkla...|
|   1006|       575|main-bonus-archiv...|
|   1007|        82|main-archive-digital|
|   1008|       573|main-news-archive...|
|   1009|       639|main-online-inter...|
|   1009|       665|main-digital-onli...|
|    101|       439|           main-news|
|   1010|       141|main-internet-arc...|
|   1011|       865|   main-rabota-vklad|
|   1012|       993|main-online-rabot...|
|   1014|       341|main-internet-tar...|
|   1015|       230|                main|
|   1018|        66|     main-bonus-news|
+-------+----------+--------------

In [85]:
result_df = (
    filtered_table.groupBy('route')
                    .agg(F.count('*').alias('count'))
                    .orderBy(F.desc('count'))
                    .select(['route', 'count'])
)

result_df.show(30)

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 8184|
|        main-archive| 1113|
|         main-rabota| 1047|
|       main-internet|  897|
|          main-bonus|  870|
|           main-news|  769|
|        main-tariffs|  677|
|         main-online|  587|
|          main-vklad|  518|
| main-rabota-archive|  170|
| main-archive-rabota|  167|
|  main-bonus-archive|  143|
|   main-rabota-bonus|  139|
|    main-news-rabota|  135|
|   main-bonus-rabota|  135|
|main-archive-inte...|  132|
|    main-rabota-news|  130|
|main-internet-rabota|  129|
|   main-archive-news|  126|
|main-rabota-internet|  124|
|main-internet-arc...|  123|
|  main-archive-bonus|  117|
| main-internet-bonus|  115|
|main-tariffs-inte...|  114|
|   main-news-archive|  113|
|  main-news-internet|  109|
|main-archive-tariffs|  104|
|  main-internet-news|  103|
|main-tariffs-archive|  103|
|    main-rabota-main|   94|
+--------------------+-----+
only showing t

In [147]:
temp = result_df.limit(30).toPandas()
with open('result_df.csv', 'w') as f:
    f.write(temp.to_csv(sep='\t', index=False))

### Solution with Spark RDD

In [100]:
clickstream_rdd = sc.textFile('hdfs:/data/clickstream.csv').map(lambda x: x.split('\t'))
clickstream_rdd.take(5)

[['user_id', 'session_id', 'event_type', 'event_page', 'timestamp'],
 ['562', '507', 'page', 'main', '1695584127'],
 ['562', '507', 'event', 'main', '1695584134'],
 ['562', '507', 'event', 'main', '1695584144'],
 ['562', '507', 'event', 'main', '1695584147']]

In [123]:
rdd = (
    clickstream_rdd
    .filter(lambda x: x[0] != 'user_id')
    .map(lambda x: ((x[0], x[1]), (x[2], x[3], int(x[4]))))
    )
rdd.take(10)

[(('562', '507'), ('page', 'main', 1695584127)),
 (('562', '507'), ('event', 'main', 1695584134)),
 (('562', '507'), ('event', 'main', 1695584144)),
 (('562', '507'), ('event', 'main', 1695584147)),
 (('562', '507'), ('wNaxLlerrorU', 'main', 1695584154)),
 (('562', '507'), ('event', 'main', 1695584154)),
 (('562', '507'), ('event', 'main', 1695584154)),
 (('562', '507'), ('event', 'main', 1695584160)),
 (('562', '507'), ('page', 'rabota', 1695584166)),
 (('562', '507'), ('event', 'rabota', 1695584174))]

In [138]:
# function to find error (if any) and get events (before error if it occured or the full list if there was not any errors)
def filter_data(data):
    sorted_events = sorted(data, key=lambda x: x[2])
    error_idx = -1
    for idx, (event_type, page, timestamp) in enumerate(sorted_events):
        if event_type.find('error') >= 0: 
            error_idx = idx
            break
    events = sorted_events[:error_idx] if error_idx != -1 else sorted_events
    return events

# create route of type page-page-page
def concat_route(pages):
    route = []
    for page in pages:
        if not route or page != route[-1]:
            route.append(page)
    return "-".join(route)

In [140]:
result_rdd = (
    clickstream_rdd
    .filter(lambda x: x[0] != 'user_id')
    .map(lambda x: ((x[0], x[1]), (x[2], x[3], int(x[4]))))
    .groupByKey()
    .map(lambda x: (x[0], filter_data(list(x[1]))))
    # .filter(lambda x: x[1] is not None)
    .map(lambda x: concat_route([event_page for _, event_page, _ in x[1]]))
    .map(lambda x: (x, 1))
    .reduceByKey(lambda a, b: a + b)
    .sortBy(lambda x: (-x[1], x[0]))
    )
result_rdd.take(30)

[('main', 8185),
 ('main-archive', 1113),
 ('main-rabota', 1047),
 ('main-internet', 897),
 ('main-bonus', 870),
 ('main-news', 769),
 ('main-tariffs', 677),
 ('main-online', 587),
 ('main-vklad', 518),
 ('main-rabota-archive', 170),
 ('main-archive-rabota', 167),
 ('main-bonus-archive', 143),
 ('main-rabota-bonus', 139),
 ('main-bonus-rabota', 135),
 ('main-news-rabota', 135),
 ('main-archive-internet', 132),
 ('main-rabota-news', 130),
 ('main-internet-rabota', 129),
 ('main-archive-news', 126),
 ('main-rabota-internet', 124),
 ('main-internet-archive', 123),
 ('main-archive-bonus', 117),
 ('main-internet-bonus', 115),
 ('main-news-archive', 113),
 ('main-tariffs-internet', 113),
 ('main-news-internet', 109),
 ('main-archive-tariffs', 104),
 ('main-internet-news', 103),
 ('main-tariffs-archive', 103),
 ('main-rabota-main', 94)]

In [146]:
(
    result_rdd
    .toDF()
    .toPandas()[:30]
    .to_csv('result_rdd.csv', sep='\t', index=False, header=['route', 'count'])
)